# Write an RNN Language Model

0. Introduction
================

The **language model** is modeling the probability of generating natural language sentences or documents. You can use the language model to estimate how natural a sentence or a document is. Also, with the language model, you can generate new sentences or documents.

Let's start with modeling the probability of generating sentences. We represent a sentence be $X = (x_0, x_1, \dots, x_T)$, in which $x_t$ is a one-hot vector.

Generally, $x_0$ is the one-hot vector of **BOS** (beginning of sentence), and $x_T$ is that  of **EOS** (end of sentence).

In the case of language model, we usually model the probability of generating the next word under the condition of the previous words. Let $X_{[i, j]}$ be $(x_i, x_{i+1}, \dots, x_j)$ , the occurrence probabilitx of sentence $X$ can be

$$P(X) = P(x_0) \prod_{t=1}^T P(x_t \mid X_{[0, t-1]})$$

As you see above, the occurrence probability $P(X)$ can be decomposed into the conditional probability by the previous words $P(x_t \mid X_{[0, t-1]})$.

In this notebook, we model the language model $P(x_t \mid X_{[0, t-1]})$ with a recurrent neural network.

1. Basic Idea of Recurrent Neural Net Language Model
=====================================================

1.1 Recurrent Neural Net Language Model
---------------------------------------

**Recurrent Neurral Net Language Model** (RNNLM) is the neural net language model which contains RNNs in the network. Since an RNN can deal with the variable length inputs, it is suitable for modeling the sequential data such that natural languages.

The probablity of generating a sentence $X$ is denoted as $P_{\rm model}(X)$,

$$P_{\rm model}(X) = P(x_0) \prod_{t=1}^T P(x_t \mid X_{[0, t-1]})$$

We show one layer of a RNNLM with these parameters.

| Symbol | Definition |
|-------:|:-----------|
| $x_t$ | the one-hot vector of $t$-th word |
| $y_t$ | the $t$-th output |
| $h_t^{(i)}$ | the $t$-th hidden layer of $i$-th layer |
| $p_t$ | the next word's probability of $t$-th word |
| $E$ | Embedding matrix |
| $W_h$ | Hidden layer matrix |
| $W_o$ | Output layer matrix |


![](rnnlm.png)

1. Get the embedding vector: $h_t^{(0)} = E x_t$
2. Calculate the hidden layer: $h_t^{(1)} = {\rm tanh}\left(W_h \left[ \begin{array}{cc} h_t^{(0)} \\ h_{t-1}^{(1)} \end{array} \right]\right)$
3. Calculate the output layer: $y_t = W_o h_t^{(1)}$
4. Transform to probability: $p_t = {\rm softmax}(y_t)$

1.1 Perplexity (Evaluation of the language model)
-----------------------------------------------

**Perplexity** is the common evaluation metric for a language model. Generally, it measures how well the proposed probability model $P_{\rm model}(X)$ predicts a separated test sample $x_1, x_2, \dots, x_N$ drawn from an unknown probability distribution $P$.

Let a test dataset be $D = \{Y^{(n)}\}_{n=1}^{|D|}$, the $n$-th sentence length be $T^{(n)}$, and the vocabulary size be $V$. Then, perplexity is represented as $b^z$, s.t.,

$$
z = - \frac{1}{V} \sum_{n=1}^{|D|} \sum_{t=1}^{T^{(n)}} \log_b P_{\rm model}(y_t^{(n)}, Y_{[a, t-1]}^{(n)})
$$

We usually use $b = 2~{\rm or}~e$. This value shows how much varied the predicted distribution for the next word is. When a language model is well fitted to the dataset, it should show high probability only for the correct next word, so that the entropy should be high. In the above equation, the sign is reversed, so that smaller perplexity means the model is better.

During training, we minimize the below cross entropy:

$$
H(\hat{P}, P_{\rm model}) = -\sum_x \hat{P}(x) \log P_{\rm model}(x)
$$

where $\hat P$ is the empirical distribution of training examples. When $x$ appeared $n$ times in $N$ training examples, $\hat P(x) = n/N$.

2. Implementation of Recurrent Neural Net Language Model
=========================================================

* There is an example related to the RNN language model in the official repository, so we will explain based on this: [chainer/examples/ptb](https://github.com/chainer/chainer/tree/master/examples/ptb)

2.1 Overview
------------

![rnnlm_example](rnnlm_example.png)

* In this example, we use the recurrent neural model above.

| Symbol | Definition |
|-------:|:-----------|
| $x_t$ | the one-hot vector of $t$-th input |
| $y_t$ | the $t$-th output |
| $h_t^{(i)}$ | the $t$-th hidden vector of $i$-th layer |
| $E$ | Embedding matrix |
| $W_o$ | Output layer matrix |

* We use **LSTM** (long short-term memory) for the connection of hidden layers. A LSTM is one of major recurrent neural net modules. It is desined for remembering the long-term memory, which means relationships of distant words, such that a word at beginning of sentence and it at the end.
* We also use **Dropout** before both LSTMs and linear transformations. Dropout is one of regularization techniques for preventing overfitting on training dataset.

2.2 Step-by-step implementation
-----------------------------

### Import Package

First, let's import necessary packages.

In [1]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions
import numpy as np

### Define training settings

In [3]:
batchsize = 20
bproplen = 1 # 35
epoch = 39
gpu = 0  # negative value to run in CPU
gradclip = 5
unit = 650
interval = 500
update_interval = 10

### Define Network Structure

In [24]:
class RNNLM(chainer.Chain):

    def __init__(self, n_vocab, n_units):
        super(RNNLM, self).__init__()
        with self.init_scope():
            self.embed = L.EmbedID(n_vocab, n_units)
            self.l1 = L.LSTM(n_units, n_units)
            self.l2 = L.LSTM(n_units, n_units)
            self.l3 = L.Linear(n_units, n_vocab)

        for param in self.params():
            param.data[...] = np.random.uniform(-0.1, 0.1, param.data.shape)

    def reset_state(self):
        self.l1.reset_state()
        self.l2.reset_state()

    def __call__(self, x):
        h0 = self.embed(x)
        h1 = self.l1(F.dropout(h0))
        h2 = self.l2(F.dropout(h1))
        y = self.l3(F.dropout(h2))
        return y

* When we insatantiate this class for making a model, we give the vocavulary size to `n_vocab` and the size of hidden vectors to `n_units`.
* This network uses `chainer.links.LSTM`, `chainer.links.Linear`, and `chainer.functions.dropout` as its building blocks.
* All the layers are registered and initialized in the context with `self.init_scope()`.
* You can access all the parameters in those layers by calling `self.params()`.
* The `__call__` method takes an one-hot input word vector `x`, and calculates the word probability vector for the next word by forwarding it through the nerwork, and returns the output.
* Note that the one-hot input word vector `x` is converted to an embedding vector `h0` by `self.embed(x)` at the first line of `__call__`.

### Define Iterator for Data

In [5]:
class ParallelSequentialIterator(chainer.dataset.Iterator):

    def __init__(self, dataset, batch_size, repeat=True):
        self.dataset = dataset
        
        # batch size
        self.batch_size = batch_size
        
        # Number of completed sweeps over the dataset. In this case, it is
        # incremented if every word is visited at least once after the last
        # increment.
        self.epoch = 0
        
        # True if the epoch is incremented at the last iteration.
        self.is_new_epoch = False
        self.repeat = repeat
        length = len(dataset)
        
        # Offsets maintain the position of each sequence in the mini-batch.
        self.offsets = [i * length // batch_size for i in range(batch_size)]
        
        # NOTE: this is not a count of parameter updates. It is just a count of
        # calls of `__next__`.
        self.iteration = 0
        
        # use -1 instead of None internally
        self._previous_epoch_detail = -1.

    def __next__(self):
        # This iterator returns a list representing a mini-batch. Each item
        # indicates a different position in the original sequence. Each item is
        # represented by a pair of two word IDs. The first word is at the
        # "current" position, while the second word is at the next position.
        # At each iteration, the iteration count is incremented, which pushes
        # forward the "current" position.
        length = len(self.dataset)
        if not self.repeat and self.iteration * self.batch_size >= length:
            # If not self.repeat, this iterator stops at the end of the first
            # epoch (i.e., when all words are visited once).
            raise StopIteration
        cur_words = self.get_words()
        self._previous_epoch_detail = self.epoch_detail
        self.iteration += 1
        next_words = self.get_words()

        epoch = self.iteration * self.batch_size // length
        self.is_new_epoch = self.epoch < epoch
        if self.is_new_epoch:
            self.epoch = epoch

        return list(zip(cur_words, next_words))

    @property
    def epoch_detail(self):
        # Floating point version of epoch.
        return self.iteration * self.batch_size / len(self.dataset)

    @property
    def previous_epoch_detail(self):
        if self._previous_epoch_detail < 0:
            return None
        return self._previous_epoch_detail

    def get_words(self):
        # It returns a list of current words.
        return [self.dataset[(offset + self.iteration) % len(self.dataset)]
                for offset in self.offsets]

    def serialize(self, serializer):
        # It is important to serialize the state to be recovered on resume.
        self.iteration = serializer('iteration', self.iteration)
        self.epoch = serializer('epoch', self.epoch)
        try:
            self._previous_epoch_detail = serializer(
                'previous_epoch_detail', self._previous_epoch_detail)
        except KeyError:
            # guess previous_epoch_detail for older version
            self._previous_epoch_detail = self.epoch + \
                (self.current_position - self.batch_size) / len(self.dataset)
            if self.epoch_detail > 0:
                self._previous_epoch_detail = max(
                    self._previous_epoch_detail, 0.)
            else:
                self._previous_epoch_detail = -1.

### Define Updater

In [6]:
class BPTTUpdater(training.StandardUpdater):

    def __init__(self, train_iter, optimizer, bprop_len, device):
        super(BPTTUpdater, self).__init__(
            train_iter, optimizer, device=device)
        self.bprop_len = bprop_len

    # The core part of the update routine can be customized by overriding.
    def update_core(self):
        loss = 0
        # When we pass one iterator and optimizer to StandardUpdater.__init__,
        # they are automatically named 'main'.
        train_iter = self.get_iterator('main')
        optimizer = self.get_optimizer('main')

        # Progress the dataset iterator for bprop_len words at each iteration.
        for i in range(self.bprop_len):
            # Get the next batch (a list of tuples of two word IDs)
            batch = train_iter.__next__()

            # Concatenate the word IDs to matrices and send them to the device
            # self.converter does this job
            # (it is chainer.dataset.concat_examples by default)
            x, t = self.converter(batch, self.device)

            # Compute the loss at this time step and accumulate it
            loss += optimizer.target(chainer.Variable(x), chainer.Variable(t))

        optimizer.target.cleargrads()  # Clear the parameter gradients
        loss.backward()  # Backprop
        loss.unchain_backward()  # Truncate the graph
        optimizer.update()  # Update the parameters

### Define Evaluation Function (Perplexity)

In [7]:
def compute_perplexity(result):
    result['perplexity'] = np.exp(result['main/loss'])
    if 'validation/main/loss' in result:
        result['val_perplexity'] = np.exp(result['validation/main/loss'])

### Load the Penn Tree Bank long word sequence dataset

In [14]:
train, val, test = chainer.datasets.get_ptb_words()
n_vocab = max(train) + 1 

`chainer.datasets.get_ptb_words` automatically downloads the Penn Treebank dataset. Each data contains a list of document IDs.

### Create iterators

In [19]:
train_iter = ParallelSequentialIterator(train, batchsize)
val_iter = ParallelSequentialIterator(val, 1, repeat=False)
test_iter = ParallelSequentialIterator(test, 1, repeat=False)

### Create RNN and classification model

In [25]:
rnn = RNNLM(n_vocab, unit)
model = L.Classifier(rnn)
model.compute_accuracy = False  # we only want the perplexity

* Create a recurrent neural net `rnn` and the classification model `model` by `chainer.links.Classifier`.
* `chainer.links.Classifier` computes the loss and accuracy based on a given input/label pair. To learn the RNN language model, we only need the loss (cross entropy). So, we turn off computing the accuracy. In this setting, the loss is calculated by `chainer.functions.softmax_cross_entropy`.

### Setup optimizer

In [26]:
optimizer = chainer.optimizers.SGD(lr=1.0)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.GradientClipping(gradclip))

### Setup and run trainer

In [28]:
updater = BPTTUpdater(train_iter, optimizer, bproplen, gpu)
trainer = training.Trainer(updater, (epoch, 'epoch'), out='result')

eval_model = model.copy()  # Model with shared params and distinct states                                                                                                                                          
eval_rnn = eval_model.predictor
trainer.extend(extensions.Evaluator(
    val_iter, eval_model, device=gpu,
    # Reset the RNN state at the beginning of each evaluation                                                                                                                                                      
    eval_hook=lambda _: eval_rnn.reset_state()))

trainer.extend(extensions.LogReport(postprocess=compute_perplexity, trigger=(interval, 'iteration')))
trainer.extend(extensions.PrintReport(
        ['epoch', 'iteration', 'perplexity', 'val_perplexity']
        ), trigger=(interval, 'iteration'))
trainer.extend(extensions.snapshot())
trainer.extend(extensions.snapshot_object(model, 'model_iter_{.updater.iteration}'))

trainer.run()

epoch       iteration   perplexity  val_perplexity
0           500         1180.08                     


KeyboardInterrupt: 

### Evaluate the final model.

In [30]:
eval_rnn.reset_state()
evaluator = extensions.Evaluator(test_iter, eval_model, device=gpu)
result = evaluator()
print('test perplexity:', np.exp(float(result['main/loss'])))

KeyboardInterrupt: 

2.3 Generating sentences
-----------------------

You can generate the sentence which starts with the word in the vocabulary. In this example, we generate the sentence which starts with the word **apple**. We use the script in the PTB example of the official repository.

https://github.com/chainer/chainer/tree/master/examples/ptb

In [35]:
%%bash
python gentxt.py -m model.npz -p apple

Traceback (most recent call last):
  File "gentxt.py", line 100, in <module>
    main()
  File "gentxt.py", line 59, in main
    serializers.load_npz(args.model, model)
  File "/Users/shunta/lib/chainer/chainer/serializers/npz.py", line 138, in load_npz
    with numpy.load(filename) as f:
  File "/Users/shunta/.pyenv/versions/3.5.2/lib/python3.5/site-packages/numpy/lib/npyio.py", line 370, in load
    fid = open(file, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'model.npz'


> apple a new u.s. economist with <unk> <unk> fixed more than to N the company said who is looking back to 